In [1]:
import os
import random as rnd
import pandas as pd 
import pgutils as pg
import algorithm as al
import numpy as np

In [2]:
root_dir = r'C:\Users\eleon\PycharmProjects\SM_Exam\data\yes_small'
#root_dir = './data/yes_small'

In [3]:
#test_data=[]
with open(os.path.join(root_dir, 'test.txt')) as f:
#with open(root_dir + "\\test.txt", "r") as f:

    test_data = f.readlines()
    
test_dataset = pg.data_to_list(test_data[2:])


In [4]:
#train_data=[]
with open(os.path.join(root_dir,"train.txt"), "r") as f:
    train_data = f.readlines()
    
train_dataset = pg.data_to_list(train_data[2:])

In [6]:
song_hash = pd.read_csv(os.path.join(root_dir,"song_hash.txt"), sep="\t", header=None)

In [ ]:
# setting parameters
dimension = rnd.choice([2, 5,10,25,50,100])
# regularization parameter set by cross validation
lam = rnd.choice([0.0001, 0.001, 0.01, 0.1, 1, 10, 20, 50, 100, 500, 1000])
# regularization parameter for dual point model
nu = rnd.choice([0, lam])
# threshold for landmark dimension
r = 8
# number of landmark
n_landmarks = 50
# num iteration 100 o 200
n_iter = rnd.choice([100, 200])
# tau predefined learning rate
tau = 0.5

#alleno l'algoritmo
U = al.single_point_algorithm(song_hash, dimension, train_dataset, n_iter, lam, nu, tau, n_landmarks, r)
# evaluate test performance using the average log-likelihood as metric
# it is defined as log(Pr(d_test))/n_test)
# n_test number of transition in the test set
songs = len(song_hash)
n_test = np.sum(pg.transition_count(songs,test_dataset))
#result = sum( sum(dimension.D[s, i.index()] ) for i in range(len(test_dataset)))  